In [101]:
import ete3
import pandas as pd
import numpy as np

In [20]:
new_tree = ete3.Tree("((D,F)E,(B,H)C)G;",format=8)

In [21]:
def attribute_indexes(tree):
    # First pass : attribute indexes to leaves
    index = 0
    for node in tree.traverse("postorder"):
        if node.is_leaf():
            node.add_features(index=index)
            index += 1
    # Second pass : attribute indexes to internal nodes
    for node in tree.traverse("postorder"):
        if not node.is_leaf():
            node.add_features(index=index)
            index += 1
    return tree


In [22]:
new_tree = attribute_indexes(new_tree)

In [53]:
list_names = [node.name for node in new_tree.traverse("postorder")]
list_depths = [node.get_distance(new_tree) for node in new_tree.traverse("postorder")]
list_lengths = [node.dist for node in new_tree.traverse("postorder")]
list_parent_depths = [0 if node.is_root() else node.up.get_distance(new_tree) for node in new_tree.traverse("postorder")]
list_children = [[child.name for child in node.children] for node in new_tree.traverse("postorder")]
list_indexes = [node.index for node in new_tree.traverse("postorder")]
list_children_indexes = [[child.index for child in node.children] for node in new_tree.traverse("postorder")]

In [54]:
data_dictionary = {"name":list_names, "depth":list_depths, "parent_depth":list_parent_depths, "dist":list_lengths, "children":list_children, "index":list_indexes, "children_indexes":list_children_indexes}

In [55]:
# Create dataframe with correct types
original_table = pd.DataFrame(data_dictionary)

In [56]:
original_table["name"] = original_table["name"].astype("string")

In [145]:
original_table.sort_values(by="index", inplace=True)

In [146]:
original_table

,name,depth,parent_depth,dist,children,index,children_indexes,time_cumsum
0,D,2.0,1.0,1.0,[],0,[],1.0
1,F,2.0,1.0,1.0,[],1,[],2.0
3,B,2.0,1.0,1.0,[],2,[],4.0
4,H,2.0,1.0,1.0,[],3,[],5.0
2,E,1.0,0.0,1.0,"[D, F]",4,"[0, 1]",3.0
5,C,1.0,0.0,1.0,"[B, H]",5,"[2, 3]",6.0
6,G,0.0,0.0,0.0,"[E, C]",6,"[4, 5]",6.0


1. Créer la subdivision temporelle
2. Mesurer la longueur des intervalles
3. Créer un index des noeuds existants sur chaque subdivision

In [61]:
def set_depths(tree_df):
    print("Attention : la fonction set_depths peut compter deux fois le même intervalle du fait d'une erreur de calcul de Tree.get_distance")
    return sorted(list(set(tree_df["depth"])))

In [63]:
def create_subdivision(depths):
    return [0] + [depths[i]-depths[i-1] for i in range(1,len(depths))]

On doit créer une table de contemporanéité avec les subdivisions. Pour chaque indice de la subdivision, on enregistre l'ensemble des indices de noeuds contenus dans cette subdivision.

In [73]:
def is_contained(number, low, high, shift = 1e-3):
    """Si un noeud est de profondeur juste au dessus de la limite supérieure, cela signifie qu'il n'est pas vraiment contenu dans l'intervalle, mais dans l'intervalle du dessus."""
    return (number >= low + shift) and (number <= high + shift)

In [88]:
depths = set_depths(original_table)
subdivision = create_subdivision(depths)

Attention : la fonction set_depths peut compter deux fois le même intervalle du fait d'une erreur de calcul de Tree.get_distance


In [85]:
list_contemporaries = [[] for i in range(len(depths))]

In [86]:
for depth in range(len(depths)-1):
    """Si un noeud est de profondeur comprise entre depths[depth] et depths[depth+1], il recouvre l'intervalle d'indice depth.
    Exemple : on a des profondeurs 0, 1, 2, 3
    Alors les intervalles sont 0, 1, 1, 1 en ajoutant l'intervalle nul de la racine. 
    Un noeud de profondeur 1 est contenu entre les profondeurs 0 et 1, mais pas entre les profondeurs 1 et 2 grâce au shift de la fonction is_contained.
    Il recouvre donc l'intervalle d'indice 1, qui est de longueur 1
    
    Un noeud de profondeur 3 est contenu entre les profondeurs 2 et 3, donc il recouvre l'intervalle d'indice 3, qui est de longueur 1.

    On ignore le noeud de profondeur 0, qui est la racine, c'est pourquoi on peut itérer sur len(depths)-1 valeurs.

    Ainsi, la liste de contemporains ne contient pas l'indice de la racine.
    """


    for index, row in original_table.iterrows():
        if is_contained(row["depth"], depths[depth], depths[depth+1]):
            list_contemporaries[depth+1].append(row["index"])

In [87]:
list_contemporaries

[[], [4, 5], [0, 1, 2, 3]]

In [89]:
contemporaneity_df = pd.DataFrame({"subdivision":subdivision, "contemporaries":list_contemporaries})

In [93]:
contemporaneity_df["nb_contemporaries"] = contemporaneity_df["contemporaries"].apply(lambda x : len(x))

In [95]:
contemporaneity_df["length_prod"] = contemporaneity_df["subdivision"] * contemporaneity_df["nb_contemporaries"]

In [97]:
contemporaneity_df["normalized_prob"] = contemporaneity_df["length_prod"] / contemporaneity_df["length_prod"].sum()

In [99]:
contemporaneity_df["prob_cumsum"] = contemporaneity_df["normalized_prob"].cumsum()

In [109]:
contemporaneity_df

,subdivision,contemporaries,nb_contemporaries,length_prod,normalized_prob,prob_cumsum
0,0.0,[],0,0.0,0.000000,0.000000
1,1.0,"[4, 5]",2,2.0,0.333333,0.333333
2,1.0,"[0, 1, 2, 3]",4,4.0,0.666667,1.000000


In [112]:
original_table

,name,depth,parent_depth,dist,children,index,children_indexes
0,D,2.0,1.0,1.0,[],0,[]
1,F,2.0,1.0,1.0,[],1,[]
2,E,1.0,0.0,1.0,"[D, F]",4,"[0, 1]"
3,B,2.0,1.0,1.0,[],2,[]
4,H,2.0,1.0,1.0,[],3,[]
5,C,1.0,0.0,1.0,"[B, H]",5,"[2, 3]"
6,G,0.0,0.0,0.0,"[E, C]",6,"[4, 5]"


Pour choisir les temps de don, on peut faire une cumsum des longueurs de branches, et choisir ainsi les temps de don

In [113]:
original_table["time_cumsum"] = original_table["dist"].cumsum()

In [115]:
original_table # Exclure la racine comme donneur potentiel ? Non ! Il faut juste mettre la racine en première, mais sa présence est importante pour choisir le temps de don

In [130]:
index_of_depth_zero = original_table[original_table["depth"] == 0].index[0]
concat_df = pd.concat([original_table.iloc[[index_of_depth_zero]], original_table.drop(index_of_depth_zero)]).reset_index(drop=True)


In [132]:
concat_df["time_cumsum"] = concat_df["dist"].cumsum()

In [133]:
concat_df

,name,depth,parent_depth,dist,children,index,children_indexes,time_cumsum
0,G,0.0,0.0,0.0,"[E, C]",6,"[4, 5]",0.0
1,D,2.0,1.0,1.0,[],0,[],1.0
2,F,2.0,1.0,1.0,[],1,[],2.0
3,E,1.0,0.0,1.0,"[D, F]",4,"[0, 1]",3.0
4,B,2.0,1.0,1.0,[],2,[],4.0
5,H,2.0,1.0,1.0,[],3,[],5.0
6,C,1.0,0.0,1.0,"[B, H]",5,"[2, 3]",6.0


In [140]:
random_number = np.random.uniform(0, 6)

# Find the index of the row with the smallest cumulative sum greater than the random number
donor = concat_df[concat_df["time_cumsum"] > random_number].min()

In [143]:
concat_df[concat_df["time_cumsum"] > random_number]["time_cumsum"].idxmin()

3

Pour pouvoir choisir aléatoirement un point de départ de chaque noeud

In [ ]:
rust_table = original_table["index", "depth", "parent_depth", "children_indexes"]